# Vectara Vector Store
In this notebook we are going to show how to use [Vectara](https://vectara.com) with LlamaIndex.
Vectara is the first example of a "Managed" Index, a new type of index in Llama-index which is managed via an API.

In [2]:
from llama_index import SimpleDirectoryReader
from llama_index.indices.managed.vectara.base import VectaraIndex
from llama_index.indices.managed.vectara.retriever import VectaraQuery

import textwrap

### Loading documents
Load the documents stored in the `paul_graham_essay` using the SimpleDirectoryReader

In [3]:
documents = SimpleDirectoryReader("../data/paul_graham").load_data()
print("Document ID:", documents[0].doc_id)

Document ID: 96cb1b28-03dc-442c-94a9-6a6749310702


### Add the content of the documents into a pre-created Vectara corpus
Here we assume an empty corpus is created and the details are available as environment variables:
* VECTARA_CORPUS_ID
* VECTARA_CUSTOMER_ID
* VECTARA_API_KEY

In [4]:
index = VectaraIndex()
for doc in documents:
    index.insert(doc)

### Query the Vectara Index
We can now ask questions using the VectaraIndex retriever.

In [15]:
q = VectaraQuery(
    query_str="who is the author?",
    similarity_top_k=10,
)   
query_engine = index.as_query_engine()
response = query_engine.query(q)

In [16]:
print(textwrap.fill(str(response), 100))

 The author is Paul Graham.


In [17]:
q = VectaraQuery(
    query_str="What did the author do growing up?",
    similarity_top_k=10,
)
response = query_engine.query(q)

In [18]:
print(textwrap.fill(str(response), 100))

 The author grew up writing short stories and programming. They wrote stories with strong feelings,
but with hardly any plot. They also worked on programming projects outside of school.
